# Desafio Escale

Este <i>notebook</i> traz a resolução do desafio proposto pela Escale (https://escaletech.github.io/dataplatform/data-engineer-test).

A <i>storytelling</i> será mais branda, pois esta modelagem será aplicado em todo o dataset, e para isso serão criados funcoes que serão chamadas em loop.

Instalação e importação de bibliotecas.

In [1]:
try:
    !pip install pyspark=="2.4.5" --quiet
    !pip install pandas=="1.0.4" --quiet
except:
    print("Running throw py file.")

In [2]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark import SparkFiles
from pyspark.sql.types import StringType, FloatType
import pyspark
import json
import pandas as pd
import numpy as np

Criação de uma sessão Spark

In [3]:
spark = SparkSession\
        .builder\
        .appName("Desafio Data Engineer Escale - Fabio Kfouri")\
        .getOrCreate()
spark

Para otimizar a resolução, foram realizados downloads dos datasets, esta lógica é para identificar se este notebook esta rodando máquina do autor, caso positivo, utilizará o dataset local, do contrário, utilizará o dataset da núvem.

In [4]:
import os

dataPath = 'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/'

if 'E:\\' in os.getcwd() and 'dataEngineerTest_Escale' in os.getcwd():
    dataPath = os.getcwd() + "/data/"

print(dataPath)


E:\Projetos\Jobs\dataEngineerTest_Escale/data/


Funcão de leitura do Json que retorna um Datafame Spark

In [5]:
def read_json(filename):
    return spark.read.json(dataPath + filename)

Função que converte o 30 minutos em miliegundos para ser usado como referência de delta para data em formado de <b>Epoch Time</b>.

In [6]:
def get_epoch_time_limit():
    return (30*60)*1000

Função que cria o dataFrame <b>ClickStream</b>, que atende os criterios de sessionamento defindo pelo desafio.

Para calculo do tempo de sessão foi utilizado a função <b>LAG()</b> que tem por objetivo trazer no registro corrente o dado de timestamp do registro anterior.

O autor assumiu como premissa não declarada para definir uma sessão, que além do tempo limite de 30 minutos desde a última utilização, que uma sessão precisaria considerar o <b>device_family</b> e <b>os_family</b>.

Ou seja, mesmo que não tenha excedido o tempo limite de 30 minutos, mas se for caracterizado que houve uma mudança de device_family ou os_family, trata-se de uma sessão nova.

Para a criação das Sessions_ID foi usado as funções ROW_NUMBER() para criacão ordenada de ids, e em seguida utilizada a função LAST_VALUE() para o preenchimento do session_id nos eventos da mesma sessão.

In [7]:
def get_clickstream_dataframe(df):
    overCategory = Window.partitionBy('anonymous_id','device_family','os_family','browser_family')\
                         .orderBy('anonymous_id','device_family','os_family','browser_family','device_sent_timestamp')
    
    time_limit = get_epoch_time_limit()
    
    df = df.withColumn("lag", F.lag('device_sent_timestamp', 1).over(overCategory))\
            .withColumn('delta_seg', (F.col('device_sent_timestamp') - F.col('lag'))/1000)\
            .withColumn('same_section', (F.col('device_sent_timestamp') - F.col('lag')) < time_limit) \
            .withColumn('event_time', (F.col('device_sent_timestamp')/1000).cast('timestamp'))
    
    df.createOrReplaceTempView("raw_table")
    
    df_sessioned = spark.sql("""
        with temp as (--
              SELECT anonymous_id, browser_family, device_family, device_sent_timestamp, event, event_time, n, os_family, --
                     platform, nvl(same_section,false) same_section, version,
                     case when nvl(same_section,false) = false then
                         0
                     else
                        delta_seg
                     end delta_seg,             
                     case when nvl(same_section,false) = false then
                     'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id,nvl(same_section,false) ORDER BY device_sent_timestamp )
                     else
                      null
                     end
                     as partial_session
                FROM raw_table t --
        ), table_section_id as (--
            select anonymous_id, browser_family, delta_seg, device_family, device_sent_timestamp, event, event_time, n, os_family, --
                     platform, nvl(same_section,false) same_section, version,
                     LAST_VALUE(partial_session,True) OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp ) || '_' || anonymous_id session_id
            from temp t
        )
        select * from table_section_id
        order by anonymous_id, device_sent_timestamp 
        """)
        
    df_sessioned.createOrReplaceTempView("clickstream")
    
    #Remove the view raw_table
    spark.catalog.dropTempView('raw_table')


Funcao que convert as saídas da Questão 2 no formato desejado pelo Desafio.

Esta função cria um dicionário que de forma incremental, vai adicionando as quantidades conforme forem sendo carregados novos datasets.

In [8]:
def convert_to_Json_challenge2(dados_pd, question):   
    for index, row in dados_pd.iterrows():
        items = str(row["collection"]).split(',')

        if not row["what"] in question:
            question[row["what"]] = {}

        for item in items:
            element = str(item).split(':')

            if not element[0] in question[row["what"]]:
                question[row["what"]][element[0]] = 0

            if len(element) > 1:
                question[row["what"]][element[0]] = float(element[1]) + question[row["what"]][element[0]]
            else:
                question[row["what"]][element[0]] = float(0) + question[row["what"]][element[0]]

                
    return question

Funcao que convert as saídas da Questão 3 no formato desejado pelo Desafio.

Esta função cria um dicionário que de forma incremental, vai calculando a mediana das medianas conforme forem sendo carregados novos datasets.

In [9]:
def convert_to_Json_challenge3(dados_pd, question):   
    for index, row in dados_pd.iterrows():

        items = str(row["collection"]).split(',')

        if not row["what"] in question:
            question[row["what"]] = {}

        for item in items:
            element = str(item).split(':')

            if not element[0] in question[row["what"]]:
                question[row["what"]][element[0]] = 0


            if len(element) > 1:
                question[row["what"]][element[0]] = np.median([ float(element[1]), question[row["what"]][element[0]] ])
            else:
                question[row["what"]][element[0]] = question[row["what"]][element[0]]

            
    return question


## Desafio 1

Calcular a quantidade total de sessões únicas por arquivo do conjunto de dados e apresentar no formato JSON.

Nesta função foi realizado um filtro para desconsiderar as sessoes abertas para descobrir o total de sessoes únicas por dataset.

In [10]:
def first_challenge(filename, question1):
    df_question_1 = spark.sql("""
        SELECT COUNT(session_id) qtd_session
        FROM clickstream
        where same_section = false
        """)
    question1[filename] = df_question_1.collect()[0]['qtd_session']
    return question1

# Desafio 2
Calcular a quantidade de sessões únicas que ocorreram em cada Browser, Sistema Operacional e Dispositivo dentro de todo o conjunto de dados.

Nesta função foi identificado as famílias (<b>browser_family</b>, <b>os_family</b> e <b>device_family</b>) e em seguida por quantidade de tipo de cada família. 

Estáo sendo consideradas somente as sessões únicas.

In [11]:
def second_challenge(question2):
    df_question_2 = spark.sql("""
        with table_temp as (--
            SELECT *
            FROM clickstream
            where same_section = false
        ), 
        table_union as (--
            SELECT 'device_family' what, device_family ref, COUNT(session_id)  qtd_session
              FROM table_temp
             group by device_family
            union
            SELECT 'os_family' what, os_family ref, COUNT(session_id)  qtd_session
              FROM table_temp
            group by os_family
            union
            SELECT 'browser_family' what, browser_family ref, COUNT(session_id)  qtd_session
              FROM table_temp
             group by browser_family --
        ),
        table_collection as (--
            select what, nvl(ref, 'Not Identified') || ':' || qtd_session collection 
            from table_union
            order by what, ref--
        )
        select what, array_join(collect_list(trim(collection)),',')  collection
        from table_collection
        group by what
        """)
    
    dados2 = df_question_2.toPandas()
    return convert_to_Json_challenge2(dados2, question2)

## Desafio 3

Calcular a mediana da duração (em segundos) entre todas sessões únicas para cada segmento.

De forma análoga ao desafio 2, esta função foram identificado as quantidades por família (<b>browser_family</b>, <b>os_family</b> e <b>device_family</b>).

Estáo sendo consideradas somente as sessões que ficaram abertas.

In [12]:
def third_challenge(question3):
    df_question_3 = spark.sql("""
        with table_temp as (--
            SELECT *
            FROM clickstream
            where same_section = true
        ), 
        table_union as (--
            SELECT 'device_family' what, device_family ref, percentile_approx(delta_seg , 0.5) median   
              FROM table_temp
             group by device_family
            union
            SELECT 'os_family' what, os_family ref, percentile_approx(delta_seg , 0.5) median 
              FROM table_temp
            group by os_family
            union
            SELECT 'browser_family' what, browser_family ref, percentile_approx(delta_seg , 0.5) median 
              FROM table_temp
             group by browser_family --
        ),
        table_collection as (--
            select what, nvl(ref, 'Not Identified') || ':' || median collection 
              from table_union
             order by what, ref--
        )
        select what, array_join(collect_list(trim(collection)),',')  collection
          from table_collection
        group by what
        """)
    
    dados3 = df_question_3.toPandas()
    return convert_to_Json_challenge3(dados3, question3)

In [13]:
def process_dataset(filename, questions):
    question1, question2, question3 = questions
    df = read_json(filename)
    get_clickstream_dataframe(df)
    question1 = first_challenge(filename, question1)
    question2 = second_challenge(question2)
    question3 = third_challenge(question3)
    
    return question1, question2, question3


In [14]:
from datetime import datetime 

quantity_of_datasets = 10
questions = {}, {}, {}

for i in range(quantity_of_datasets):
    filename = 'part-0000' + str(i) +'.json.gz'
    print(filename, 'Start at', datetime.today())
    questions = process_dataset(filename, questions)
    #spark.sparkContext.addFile(dataPath + 'part-0000' + str(i) +'.json.gz')

question1, question2, question3 = questions

print('Finished at', datetime.today())

part-00000.json.gz Start at 2020-08-28 22:16:00.113178
{'part-00000.json.gz': 10234891}
{'device_family': {'': 88.0, '1001-G Go': 221.0, '2014819': 69.0, '4009E': 5.0, '4009I': 6.0, '4017F': 231.0, '4028E': 2.0, '4034E': 1502.0, '4055J': 170.0, '5010E': 1464.0, '5016J': 85.0, '5017E': 9.0, '5026J': 288.0, '5033E': 395.0, '5033J': 856.0, '5045J': 489.0, '5046J': 95.0, '5049E': 26.0, '5051J': 247.0, '5054W': 11.0, '5056N': 29.0, '5085J': 223.0, '5085N': 515.0, '5090I': 25.0, '5152D': 360.0, '5159J': 963.0, '5186D': 156.0, '5199I': 77.0, '6039J': 26.0, '6055B': 224.0, '6060S': 9.0, '62S': 56.0, '7048A': 2.0, '705-G': 22.0, '705-G Go': 488.0, '71S': 52.0, '8050E': 1717.0, '9008J': 397.0, '9008N': 673.0, '91S': 2.0, 'A3_Pro': 8.0, 'ALE-L21': 81.0, 'ATU-LX3': 1.0, 'Advance 4.0 L3': 36.0, 'Advance 4.0M': 42.0, 'Advance 5.2': 16.0, 'Advance L4': 80.0, 'Aquaris X Pro': 18.0, 'Archos 60 Platinum': 21.0, 'Archos Access 50 Color 3G': 26.0, 'Archos Core 50 4G': 70.0, 'Armor_3': 1.0, 'Armor_6': 20.0

In [16]:
print(question1)

{'part-00000.json.gz': 10234891}


In [17]:
print(question2)

{'device_family': {'': 88.0, '1001-G Go': 221.0, '2014819': 69.0, '4009E': 5.0, '4009I': 6.0, '4017F': 231.0, '4028E': 2.0, '4034E': 1502.0, '4055J': 170.0, '5010E': 1464.0, '5016J': 85.0, '5017E': 9.0, '5026J': 288.0, '5033E': 395.0, '5033J': 856.0, '5045J': 489.0, '5046J': 95.0, '5049E': 26.0, '5051J': 247.0, '5054W': 11.0, '5056N': 29.0, '5085J': 223.0, '5085N': 515.0, '5090I': 25.0, '5152D': 360.0, '5159J': 963.0, '5186D': 156.0, '5199I': 77.0, '6039J': 26.0, '6055B': 224.0, '6060S': 9.0, '62S': 56.0, '7048A': 2.0, '705-G': 22.0, '705-G Go': 488.0, '71S': 52.0, '8050E': 1717.0, '9008J': 397.0, '9008N': 673.0, '91S': 2.0, 'A3_Pro': 8.0, 'ALE-L21': 81.0, 'ATU-LX3': 1.0, 'Advance 4.0 L3': 36.0, 'Advance 4.0M': 42.0, 'Advance 5.2': 16.0, 'Advance L4': 80.0, 'Aquaris X Pro': 18.0, 'Archos 60 Platinum': 21.0, 'Archos Access 50 Color 3G': 26.0, 'Archos Core 50 4G': 70.0, 'Armor_3': 1.0, 'Armor_6': 20.0, 'Asus A001D': 10748.0, 'Asus A007': 1883.0, 'Asus ASUS': 47.0, 'Asus FBBD': 300.0, 'As

In [18]:
print(question3)

{'device_family': {'Generic Smartphone': 316.893, 'Samsung SM-A105FN': 400.442, 'Samsung SM-G9650': 794.2095, 'Samsung SM-J415G': 403.402, 'Spider': 201.988, 'iPhone': 97.544}, 'os_family': {'Android': 292.9295, 'Other': 180.021, 'iOS': 97.544}, 'browser_family': {'Chrome Mobile': 403.402, 'Facebook': 400.442, 'Googlebot': 201.988, 'Mobile Safari': 97.544}}
